In [24]:
import pandas as pd
import numpy as np
import re
import string
import emoji
import nltk
from nltk.corpus import stopwords

# Download stopwords (only the first time)
nltk.download('stopwords')

# Prepare stopwords set
#stop_words = set(stopwords.words('english'))
#from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to C:\Users\TUF
[nltk_data]     GAMING\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
# Load your dataset
df = pd.read_csv("E:\\saas-idea-validator\\data\\raw\\raw_marged\\merged_output.csv")

# Check first few rows
df.head(10)


,post_id,title,text,author,created_utc,num_comments,upvotes,upvote_ratio,post_age_days,avg_comment_recency,validation_score,label,source_url
0,1oax4zh,MoodLift AI 🚀 – Generates personalized motivat...,MoodLift AI 🚀 – Generates personalized motivat...,Cyberkukro,2025-10-19T18:29:28Z,0,1,1.0,0,0.0,55.0,neutral,https://reddit.com/r/SaaS/comments/1oax4zh/moo...
1,1oax134,[SaaS Exit] Just listed my product on Acquire—...,[SaaS Exit] Just listed my product on Acquire—...,CoolDiscipline9842,2025-10-19T18:25:16Z,0,1,1.0,0,0.0,55.0,neutral,https://reddit.com/r/SaaS/comments/1oax134/saa...
2,1oawwms,What If Your CRM Could Think Like a Salesperson?,What If Your CRM Could Think Like a Salesperso...,ValueThreader,2025-10-19T18:20:30Z,0,1,1.0,0,0.0,55.0,neutral,https://reddit.com/r/SaaS/comments/1oawwms/wha...
3,1oawrz8,Trying to make a simple CRM - open to any feed...,Trying to make a simple CRM - open to any feed...,sanxofr,2025-10-19T18:15:40Z,0,1,1.0,0,0.0,55.0,neutral,https://reddit.com/r/SaaS/comments/1oawrz8/try...
4,1oawogv,Got rejected by an accelerator. It turned out ...,Got rejected by an accelerator. It turned out ...,razmaztazz,2025-10-19T18:12:01Z,2,2,1.0,0,1.0,85.0,good,https://reddit.com/r/SaaS/comments/1oawogv/got...
5,1oawnsa,I wish I could have something that shows me al...,I wish I could have something that shows me al...,BestWatercress1690,2025-10-19T18:11:20Z,0,1,1.0,0,0.0,55.0,neutral,https://reddit.com/r/SaaS/comments/1oawnsa/i_w...
6,1oaw427,Kickstarter success?,Kickstarter success? Hi everyone. We've built ...,checkout2359,2025-10-19T17:50:14Z,0,1,1.0,0,0.0,55.0,neutral,https://reddit.com/r/SaaS/comments/1oaw427/kic...
7,1oaw3ux,Hello can you test this app Shofans-Platform ...,Hello can you test this app Shofans-Platform ...,Special_Adeptness761,2025-10-19T17:50:02Z,0,1,1.0,0,0.0,55.0,neutral,https://reddit.com/r/SaaS/comments/1oaw3ux/hel...
8,1oaw0gq,Full list of AI Software and Compute Grants an...,Full list of AI Software and Compute Grants an...,3niti14045,2025-10-19T17:46:22Z,0,1,1.0,0,0.0,55.0,neutral,https://reddit.com/r/SaaS/comments/1oaw0gq/ful...
9,1oavyej,"Hey guys,","Hey guys, I want to build some software, but b...",Accomplished-Fig1910,2025-10-19T17:44:10Z,2,1,1.0,0,1.0,85.0,good,https://reddit.com/r/SaaS/comments/1oavyej/hey...


In [26]:
# Keep relevant columns only
df = df[['title', 'text', 'num_comments', 'upvotes', 'upvote_ratio', 'post_age_days', 
         'avg_comment_recency', 'validation_score', 'label']]
df.head(10)

,title,text,num_comments,upvotes,upvote_ratio,post_age_days,avg_comment_recency,validation_score,label
0,MoodLift AI 🚀 – Generates personalized motivat...,MoodLift AI 🚀 – Generates personalized motivat...,0,1,1.0,0,0.0,55.0,neutral
1,[SaaS Exit] Just listed my product on Acquire—...,[SaaS Exit] Just listed my product on Acquire—...,0,1,1.0,0,0.0,55.0,neutral
2,What If Your CRM Could Think Like a Salesperson?,What If Your CRM Could Think Like a Salesperso...,0,1,1.0,0,0.0,55.0,neutral
3,Trying to make a simple CRM - open to any feed...,Trying to make a simple CRM - open to any feed...,0,1,1.0,0,0.0,55.0,neutral
4,Got rejected by an accelerator. It turned out ...,Got rejected by an accelerator. It turned out ...,2,2,1.0,0,1.0,85.0,good
5,I wish I could have something that shows me al...,I wish I could have something that shows me al...,0,1,1.0,0,0.0,55.0,neutral
6,Kickstarter success?,Kickstarter success? Hi everyone. We've built ...,0,1,1.0,0,0.0,55.0,neutral
7,Hello can you test this app Shofans-Platform ...,Hello can you test this app Shofans-Platform ...,0,1,1.0,0,0.0,55.0,neutral
8,Full list of AI Software and Compute Grants an...,Full list of AI Software and Compute Grants an...,0,1,1.0,0,0.0,55.0,neutral
9,"Hey guys,","Hey guys, I want to build some software, but b...",2,1,1.0,0,1.0,85.0,good


In [27]:
# Combine title and text
df['content'] = df['title'].fillna('') + ' ' + df['text'].fillna('')
df.drop(['title', 'text'], axis=1, inplace=True)
df.head(10)

,num_comments,upvotes,upvote_ratio,post_age_days,avg_comment_recency,validation_score,label,content
0,0,1,1.0,0,0.0,55.0,neutral,MoodLift AI 🚀 – Generates personalized motivat...
1,0,1,1.0,0,0.0,55.0,neutral,[SaaS Exit] Just listed my product on Acquire—...
2,0,1,1.0,0,0.0,55.0,neutral,What If Your CRM Could Think Like a Salesperso...
3,0,1,1.0,0,0.0,55.0,neutral,Trying to make a simple CRM - open to any feed...
4,2,2,1.0,0,1.0,85.0,good,Got rejected by an accelerator. It turned out ...
5,0,1,1.0,0,0.0,55.0,neutral,I wish I could have something that shows me al...
6,0,1,1.0,0,0.0,55.0,neutral,Kickstarter success? Kickstarter success? Hi e...
7,0,1,1.0,0,0.0,55.0,neutral,Hello can you test this app Shofans-Platform ...
8,0,1,1.0,0,0.0,55.0,neutral,Full list of AI Software and Compute Grants an...
9,2,1,1.0,0,1.0,85.0,good,"Hey guys, Hey guys, I want to build some softw..."


In [28]:
df['text_no_url'] = df['content'].apply(lambda x: re.sub(r'http\S+|www\S+', '', str(x)))
df[['content', 'text_no_url']].head(10)


,content,text_no_url
0,MoodLift AI 🚀 – Generates personalized motivat...,MoodLift AI 🚀 – Generates personalized motivat...
1,[SaaS Exit] Just listed my product on Acquire—...,[SaaS Exit] Just listed my product on Acquire—...
2,What If Your CRM Could Think Like a Salesperso...,What If Your CRM Could Think Like a Salesperso...
3,Trying to make a simple CRM - open to any feed...,Trying to make a simple CRM - open to any feed...
4,Got rejected by an accelerator. It turned out ...,Got rejected by an accelerator. It turned out ...
5,I wish I could have something that shows me al...,I wish I could have something that shows me al...
6,Kickstarter success? Kickstarter success? Hi e...,Kickstarter success? Kickstarter success? Hi e...
7,Hello can you test this app Shofans-Platform ...,Hello can you test this app Shofans-Platform ...
8,Full list of AI Software and Compute Grants an...,Full list of AI Software and Compute Grants an...
9,"Hey guys, Hey guys, I want to build some softw...","Hey guys, Hey guys, I want to build some softw..."


In [29]:
df['text_no_emoji'] = df['text_no_url'].apply(lambda x: emoji.replace_emoji(x, replace=''))
df[['text_no_url', 'text_no_emoji']].head(10)


,text_no_url,text_no_emoji
0,MoodLift AI 🚀 – Generates personalized motivat...,MoodLift AI – Generates personalized motivati...
1,[SaaS Exit] Just listed my product on Acquire—...,[SaaS Exit] Just listed my product on Acquire—...
2,What If Your CRM Could Think Like a Salesperso...,What If Your CRM Could Think Like a Salesperso...
3,Trying to make a simple CRM - open to any feed...,Trying to make a simple CRM - open to any feed...
4,Got rejected by an accelerator. It turned out ...,Got rejected by an accelerator. It turned out ...
5,I wish I could have something that shows me al...,I wish I could have something that shows me al...
6,Kickstarter success? Kickstarter success? Hi e...,Kickstarter success? Kickstarter success? Hi e...
7,Hello can you test this app Shofans-Platform ...,Hello can you test this app Shofans-Platform ...
8,Full list of AI Software and Compute Grants an...,Full list of AI Software and Compute Grants an...
9,"Hey guys, Hey guys, I want to build some softw...","Hey guys, Hey guys, I want to build some softw..."


In [30]:
df['text_no_punct'] = df['text_no_emoji'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
df[['text_no_emoji', 'text_no_punct']].head(10)


,text_no_emoji,text_no_punct
0,MoodLift AI – Generates personalized motivati...,MoodLift AI – Generates personalized motivati...
1,[SaaS Exit] Just listed my product on Acquire—...,SaaS Exit Just listed my product on Acquire—he...
2,What If Your CRM Could Think Like a Salesperso...,What If Your CRM Could Think Like a Salesperso...
3,Trying to make a simple CRM - open to any feed...,Trying to make a simple CRM open to any feedb...
4,Got rejected by an accelerator. It turned out ...,Got rejected by an accelerator It turned out t...
5,I wish I could have something that shows me al...,I wish I could have something that shows me al...
6,Kickstarter success? Kickstarter success? Hi e...,Kickstarter success Kickstarter success Hi eve...
7,Hello can you test this app Shofans-Platform ...,Hello can you test this app ShofansPlatform ...
8,Full list of AI Software and Compute Grants an...,Full list of AI Software and Compute Grants an...
9,"Hey guys, Hey guys, I want to build some softw...",Hey guys Hey guys I want to build some softwar...


In [31]:
df['text_no_numbers'] = df['text_no_punct'].apply(lambda x: re.sub(r'\d+', '', x))
df[['text_no_punct', 'text_no_numbers']].head(10)


,text_no_punct,text_no_numbers
0,MoodLift AI – Generates personalized motivati...,MoodLift AI – Generates personalized motivati...
1,SaaS Exit Just listed my product on Acquire—he...,SaaS Exit Just listed my product on Acquire—he...
2,What If Your CRM Could Think Like a Salesperso...,What If Your CRM Could Think Like a Salesperso...
3,Trying to make a simple CRM open to any feedb...,Trying to make a simple CRM open to any feedb...
4,Got rejected by an accelerator It turned out t...,Got rejected by an accelerator It turned out t...
5,I wish I could have something that shows me al...,I wish I could have something that shows me al...
6,Kickstarter success Kickstarter success Hi eve...,Kickstarter success Kickstarter success Hi eve...
7,Hello can you test this app ShofansPlatform ...,Hello can you test this app ShofansPlatform ...
8,Full list of AI Software and Compute Grants an...,Full list of AI Software and Compute Grants an...
9,Hey guys Hey guys I want to build some softwar...,Hey guys Hey guys I want to build some softwar...


In [32]:
df['text_lower'] = df['text_no_numbers'].str.lower()
df[['text_no_numbers', 'text_lower']].head(10)


,text_no_numbers,text_lower
0,MoodLift AI – Generates personalized motivati...,moodlift ai – generates personalized motivati...
1,SaaS Exit Just listed my product on Acquire—he...,saas exit just listed my product on acquire—he...
2,What If Your CRM Could Think Like a Salesperso...,what if your crm could think like a salesperso...
3,Trying to make a simple CRM open to any feedb...,trying to make a simple crm open to any feedb...
4,Got rejected by an accelerator It turned out t...,got rejected by an accelerator it turned out t...
5,I wish I could have something that shows me al...,i wish i could have something that shows me al...
6,Kickstarter success Kickstarter success Hi eve...,kickstarter success kickstarter success hi eve...
7,Hello can you test this app ShofansPlatform ...,hello can you test this app shofansplatform ...
8,Full list of AI Software and Compute Grants an...,full list of ai software and compute grants an...
9,Hey guys Hey guys I want to build some softwar...,hey guys hey guys i want to build some softwar...


In [33]:
df['text_clean_spaces'] = df['text_lower'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())
df[['text_lower', 'text_clean_spaces']].head(10)


,text_lower,text_clean_spaces
0,moodlift ai – generates personalized motivati...,moodlift ai – generates personalized motivatio...
1,saas exit just listed my product on acquire—he...,saas exit just listed my product on acquire—he...
2,what if your crm could think like a salesperso...,what if your crm could think like a salesperso...
3,trying to make a simple crm open to any feedb...,trying to make a simple crm open to any feedba...
4,got rejected by an accelerator it turned out t...,got rejected by an accelerator it turned out t...
5,i wish i could have something that shows me al...,i wish i could have something that shows me al...
6,kickstarter success kickstarter success hi eve...,kickstarter success kickstarter success hi eve...
7,hello can you test this app shofansplatform ...,hello can you test this app shofansplatform th...
8,full list of ai software and compute grants an...,full list of ai software and compute grants an...
9,hey guys hey guys i want to build some softwar...,hey guys hey guys i want to build some softwar...


In [34]:
stop_words = set(stopwords.words('english'))
df['text_no_stopwords'] = df['text_clean_spaces'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))
df[['text_clean_spaces', 'text_no_stopwords']].head(10)


,text_clean_spaces,text_no_stopwords
0,moodlift ai – generates personalized motivatio...,moodlift ai – generates personalized motivatio...
1,saas exit just listed my product on acquire—he...,saas exit listed product acquire—heres story s...
2,what if your crm could think like a salesperso...,crm could think like salesperson crm could thi...
3,trying to make a simple crm open to any feedba...,trying make simple crm open feedback trying ma...
4,got rejected by an accelerator it turned out t...,got rejected accelerator turned best thing hap...
5,i wish i could have something that shows me al...,wish could something shows emails slack messag...
6,kickstarter success kickstarter success hi eve...,kickstarter success kickstarter success hi eve...
7,hello can you test this app shofansplatform th...,hello test app shofansplatform helps improve e...
8,full list of ai software and compute grants an...,full list ai software compute grants credits h...
9,hey guys hey guys i want to build some softwar...,hey guys hey guys want build software came val...


In [35]:
df['text_final'] = df['text_no_stopwords'].apply(lambda x: re.sub(r'[^A-Za-z\s]', '', x))

df[['text_no_stopwords', 'text_final']].head(10)


,text_no_stopwords,text_final
0,moodlift ai – generates personalized motivatio...,moodlift ai generates personalized motivation...
1,saas exit listed product acquire—heres story s...,saas exit listed product acquireheres story sa...
2,crm could think like salesperson crm could thi...,crm could think like salesperson crm could thi...
3,trying make simple crm open feedback trying ma...,trying make simple crm open feedback trying ma...
4,got rejected accelerator turned best thing hap...,got rejected accelerator turned best thing hap...
5,wish could something shows emails slack messag...,wish could something shows emails slack messag...
6,kickstarter success kickstarter success hi eve...,kickstarter success kickstarter success hi eve...
7,hello test app shofansplatform helps improve e...,hello test app shofansplatform helps improve e...
8,full list ai software compute grants credits h...,full list ai software compute grants credits h...
9,hey guys hey guys want build software came val...,hey guys hey guys want build software came val...


In [36]:
df.head(10)

,num_comments,upvotes,upvote_ratio,post_age_days,avg_comment_recency,validation_score,label,content,text_no_url,text_no_emoji,text_no_punct,text_no_numbers,text_lower,text_clean_spaces,text_no_stopwords,text_final
0,0,1,1.0,0,0.0,55.0,neutral,MoodLift AI 🚀 – Generates personalized motivat...,MoodLift AI 🚀 – Generates personalized motivat...,MoodLift AI – Generates personalized motivati...,MoodLift AI – Generates personalized motivati...,MoodLift AI – Generates personalized motivati...,moodlift ai – generates personalized motivati...,moodlift ai – generates personalized motivatio...,moodlift ai – generates personalized motivatio...,moodlift ai generates personalized motivation...
1,0,1,1.0,0,0.0,55.0,neutral,[SaaS Exit] Just listed my product on Acquire—...,[SaaS Exit] Just listed my product on Acquire—...,[SaaS Exit] Just listed my product on Acquire—...,SaaS Exit Just listed my product on Acquire—he...,SaaS Exit Just listed my product on Acquire—he...,saas exit just listed my product on acquire—he...,saas exit just listed my product on acquire—he...,saas exit listed product acquire—heres story s...,saas exit listed product acquireheres story sa...
2,0,1,1.0,0,0.0,55.0,neutral,What If Your CRM Could Think Like a Salesperso...,What If Your CRM Could Think Like a Salesperso...,What If Your CRM Could Think Like a Salesperso...,What If Your CRM Could Think Like a Salesperso...,What If Your CRM Could Think Like a Salesperso...,what if your crm could think like a salesperso...,what if your crm could think like a salesperso...,crm could think like salesperson crm could thi...,crm could think like salesperson crm could thi...
3,0,1,1.0,0,0.0,55.0,neutral,Trying to make a simple CRM - open to any feed...,Trying to make a simple CRM - open to any feed...,Trying to make a simple CRM - open to any feed...,Trying to make a simple CRM open to any feedb...,Trying to make a simple CRM open to any feedb...,trying to make a simple crm open to any feedb...,trying to make a simple crm open to any feedba...,trying make simple crm open feedback trying ma...,trying make simple crm open feedback trying ma...
4,2,2,1.0,0,1.0,85.0,good,Got rejected by an accelerator. It turned out ...,Got rejected by an accelerator. It turned out ...,Got rejected by an accelerator. It turned out ...,Got rejected by an accelerator It turned out t...,Got rejected by an accelerator It turned out t...,got rejected by an accelerator it turned out t...,got rejected by an accelerator it turned out t...,got rejected accelerator turned best thing hap...,got rejected accelerator turned best thing hap...
5,0,1,1.0,0,0.0,55.0,neutral,I wish I could have something that shows me al...,I wish I could have something that shows me al...,I wish I could have something that shows me al...,I wish I could have something that shows me al...,I wish I could have something that shows me al...,i wish i could have something that shows me al...,i wish i could have something that shows me al...,wish could something shows emails slack messag...,wish could something shows emails slack messag...
6,0,1,1.0,0,0.0,55.0,neutral,Kickstarter success? Kickstarter success? Hi e...,Kickstarter success? Kickstarter success? Hi e...,Kickstarter success? Kickstarter success? Hi e...,Kickstarter success Kickstarter success Hi eve...,Kickstarter success Kickstarter success Hi eve...,kickstarter success kickstarter success hi eve...,kickstarter success kickstarter success hi eve...,kickstarter success kickstarter success hi eve...,kickstarter success kickstarter success hi eve...
7,0,1,1.0,0,0.0,55.0,neutral,Hello can you test this app Shofans-Platform ...,Hello can you test this app Shofans-Platform ...,Hello can you test this app Shofans-Platform ...,Hello can you test this app ShofansPlatform ...,Hello can you test this app ShofansPlatform ...,hello can you test this app shofansplatform ...,hello can you test this app shofansplatform th...,hello test app shofansplatform helps improve e...,hello test app shofanspl

In [37]:
# Drop the specified columns if they exist
columns_to_drop = ['content', 'text_no_url', 'text_no_emoji', 'text_no_punct', 
                   'text_no_numbers', 'text_lower', 'text_clean_spaces', 'text_no_stopwords']
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

df.head(10)


,num_comments,upvotes,upvote_ratio,post_age_days,avg_comment_recency,validation_score,label,text_final
0,0,1,1.0,0,0.0,55.0,neutral,moodlift ai generates personalized motivation...
1,0,1,1.0,0,0.0,55.0,neutral,saas exit listed product acquireheres story sa...
2,0,1,1.0,0,0.0,55.0,neutral,crm could think like salesperson crm could thi...
3,0,1,1.0,0,0.0,55.0,neutral,trying make simple crm open feedback trying ma...
4,2,2,1.0,0,1.0,85.0,good,got rejected accelerator turned best thing hap...
5,0,1,1.0,0,0.0,55.0,neutral,wish could something shows emails slack messag...
6,0,1,1.0,0,0.0,55.0,neutral,kickstarter success kickstarter success hi eve...
7,0,1,1.0,0,0.0,55.0,neutral,hello test app shofansplatform helps improve e...
8,0,1,1.0,0,0.0,55.0,neutral,full list ai software compute grants credits h...
9,2,1,1.0,0,1.0,85.0,good,hey guys hey guys want build software came val...


In [38]:
# Save the dataframe to a new CSV file
df.to_csv('E:\\saas-idea-validator\\data\\processed\\cleaned_dataset.csv', index=False)

print("Cleaned dataset saved as 'cleaned_dataset.csv'")
df

Cleaned dataset saved as 'cleaned_dataset.csv'


,num_comments,upvotes,upvote_ratio,post_age_days,avg_comment_recency,validation_score,label,text_final
0,0,1,1.00,0,0.00,55.00,neutral,moodlift ai generates personalized motivation...
1,0,1,1.00,0,0.00,55.00,neutral,saas exit listed product acquireheres story sa...
2,0,1,1.00,0,0.00,55.00,neutral,crm could think like salesperson crm could thi...
3,0,1,1.00,0,0.00,55.00,neutral,trying make simple crm open feedback trying ma...
4,2,2,1.00,0,1.00,85.00,good,got rejected accelerator turned best thing hap...
...,...,...,...,...,...,...,...,...
6103,2,4,0.84,4,0.90,76.45,good,first app hit users building next project firs...
6104,118,33,0.97,4,0.93,79.79,good,words problem solve words problem solve
6105,85,149,0.90,4,0.98,79.91,good,mrr k months ignoring every startup guru twitt...
6106,4,18,0.96,4,0.89,78.41,good,hours working saas exact build order saved yak...
